In [16]:
import pandas as pd

import sys
sys.path.append("..")
from src.funciones import * 

# Limpieza archivo CSV (Fuente: Tripadvisor):

Una vez tenemos el archivos csv,procedemos a limpiarlo.

Tras ver el dataframe procedemos con la siguiente limpieza:

* Ponemos los títulos bien
* Quitamos las filas que han salido mal durante la extracción de datos
* Quitamos los números que preceden a los nomber
* Reseteamos los índices
* Quitamos de la columna opiniones Abierto/Cerrado ahora
* Quitamos de Tipo '€' y los añadimos a precio la equivalencia
* Quitamos Carta de Tipo

In [17]:
ruta_al_csv = '..\data\Tripadvisor2.csv'
df = pd.read_csv(ruta_al_csv)
data=df.copy()

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       281 non-null    object
 1   1       281 non-null    object
 2   2       281 non-null    object
 3   3       25 non-null     object
 4   4       2 non-null      object
dtypes: object(5)
memory usage: 11.1+ KB


In [19]:
data.describe()

,0,1,2,3,4
count,281,281,281,25,2
unique,204,206,95,9,1
top,MICHELIN,Viajes,"Italiana, Pizza€€ - €€€Carta",Iniciar sesión,Iniciar sesión
freq,14,11,29,9,2


In [20]:
data.columns=['Nombre','Opiniones','Tipo','Precio','Cosas'] # renombramos columnas

eliminar=['Patrocinado', 'Descubrir','MICHELIN','Opinión'] # quitamos las filas que se repiten y no nos interesan porque se han colado
data = data[~data['Nombre'].isin(eliminar)]

data['Nombre'] = data['Nombre'].str.replace(r'\d+\.\s', '', regex=True) #eliminamos numeros de nombre
data = data.reset_index(drop=True)

# Limpiamos loa columna Opiniones y Tipo

data['Opiniones'] = data['Opiniones'].str.replace('Abierto ahora', '')
data['Opiniones'] = data['Opiniones'].str.replace('Cerrado ahora', '')

data['Tipo'] = data['Tipo'].str.replace(r'Carta$', '', regex=True)
data['Precio'] = data['Tipo'].str.extract(r'(€+)')


def precio(tipo):
    if '€€€€' in tipo:
        return 'Alto'
    elif '€€ - €€€' in tipo:
        return 'Medio'
    elif '€' in tipo:
        return 'Bajo'
    else:
        return ''
    

data['Precio'] = data['Tipo'].apply(precio)
data['Tipo'] = data['Tipo'].str.replace('€', '', regex=False)
data['Tipo'] = data['Tipo'].str.replace('-', '', regex=False)
data.head()


limpieza_tipo(data)


In [21]:
import regex as re

cerrado = [re.sub(r'Cerrado hoy', '', cadena) for cadena in data.Opiniones]
minutos = [re.sub(r'Abre en 6 minutos', '', opinion) for opinion in data.Opiniones]

data['Opiniones'] = data['Opiniones'].apply(lambda x: re.sub(r'Abre en 6 minutos', '', x))
data['Opiniones'] = data['Opiniones'].apply(lambda x: re.sub(r'Cerrado hoy', '', x))

In [22]:
def eliminar_palabras(texto):
    palabras_a_eliminar = ["Latina", "Francesa", "Saludable", "Internacional",'rápida','saludable']
    patron = r'\b(?:' + '|'.join(palabras_a_eliminar) + r')\b'
    return re.sub(patron, '', texto)

# Aplica la función a la columna "Tipo"

data['Tipo'] = data['Tipo'].apply(eliminar_palabras)


In [23]:
data.head()

,Nombre,Opiniones,Tipo,Precio,Cosas
0,Pizzeria Fratelli,460 opiniones,Italiana,Medio,NaN
1,Asian Army,52 opiniones,Vietnamita,Medio,NaN
2,Menomale,204 opiniones,Italiana,Medio,NaN
3,Borax,445 opiniones,Caribeña,Medio,NaN
4,Éccolo,133 opiniones,Italiana,Medio,NaN


In [24]:
valores_unicos = data['Tipo'].value_counts()
valores_unicos = valores_unicos[valores_unicos == 1]

# Convierte los valores únicos en una lista
valores_unicos_lista = valores_unicos.index.tolist()
valores_unicos_lista

['Venezolana',
 'Americana ',
 'Colombiana',
 'Venezolana  ',
 'Filipina',
 'LatinaComida   ',
 'LatinaInternacional  ',
 'Comida rápidaInternacional  ',
 'FusiónSaludable  ',
 'Argentino  ',
 'Americana  ',
 'Coreana  ',
 'Turca  ']

In [25]:
eliminar=['LatinaComida   ',
 'LatinaInternacional  ',
 'Comida rápidaInternacional  ',
 'FusiónSaludable  ',]

data.index[data['Tipo'].isin(eliminar)]

Index([49, 114, 117, 122], dtype='int64')

In [26]:
data = data.drop(data.index[data['Tipo'].isin(eliminar)])

In [27]:
data.Nombre = data.Nombre.str.upper()

In [30]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 0 to 255
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nombre     252 non-null    object
 1   Opiniones  252 non-null    object
 2   Tipo       252 non-null    object
 3   Precio     252 non-null    object
 4   Cosas      0 non-null      object
dtypes: object(5)
memory usage: 77.1 KB


In [31]:
data = optim(data)

In [32]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 252 entries, 0 to 255
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Nombre     252 non-null    category
 1   Opiniones  252 non-null    category
 2   Tipo       252 non-null    category
 3   Precio     252 non-null    category
 4   Cosas      0 non-null      category
dtypes: category(5)
memory usage: 41.4 KB


In [33]:
data.to_csv('../data/Tripadvisor.csv')